This notebook is used to obtain the additional metadata information for the studies served on D2H2. There are files that are read for manual annotation of the studies as well as automatic annotation using the GEO studes supplementary information. The additional metadata includes the tissue types, disease types, platform types, and perturbations for each study. The output is a dictionary that stores the metadata for each study within each species. In addition, a `metadata_counts.json` file is outputted that will be used to popualte the about page with the statisical distributions for the various metadata collected. 

In [11]:
import pandas as pd
import GEOparse
import os
import json
import ftfy
from collections import defaultdict 
import s3fs
import os
import json
import pandas as pd
from dotenv import load_dotenv
load_dotenv()


True

## Initializing variables for the metadata analysis

In [12]:

endpoint = os.environ.get('ENDPOINT', 'https://d2h2.s3.amazonaws.com/')

#USE THE BELOW IF YOU HAVE A KEY AND SECRET VARIABLE
base_url = os.environ.get('BASE_URL', 'd2h2/data')
s3 = s3fs.S3FileSystem(key = os.environ.get('AWS_ACCESS_KEY_ID'), secret = os.environ.get('AWS_SECRET_ACCESS_KEY'))
print(s3.ls('d2h2'))
print(len(list(s3.walk(f'{base_url}/mouse', maxdepth=1))[0][1]))
print(len(list(s3.walk(f'{base_url}/human', maxdepth=1))[0][1]))
print(len(list(s3.walk(f'{base_url}/mouse_single', maxdepth=1))[0][1]))
print(len(list(s3.walk(f'{base_url}/human_single', maxdepth=1))[0][1]))
dict_storing_gses = {'mouse':list(s3.walk(f'{base_url}/mouse', maxdepth=1))[0][1], 'human': list(s3.walk(f'{base_url}/human', maxdepth=1))[0][1],
                      'mouse_single': list(s3.walk(f'{base_url}/mouse_single', maxdepth=1))[0][1],
                      'human_single': list(s3.walk(f'{base_url}/human_single', maxdepth=1))[0][1]}




['d2h2/data', 'd2h2/t2d-datasets']
145
145
30
17


In [13]:
dict_storing_information = {'human':{'assay': defaultdict(int), 'disease': defaultdict(int), 'tissue': defaultdict(int), 'platform': defaultdict(int)},
                            'mouse':{'assay': defaultdict(int), 'disease': defaultdict(int), 'tissue': defaultdict(int), 'platform': defaultdict(int)}}


In [14]:
dict_storing_information['human_single'] = {'assay': defaultdict(int), 'disease': defaultdict(int),
                                            'tissue': defaultdict(int), 'cell type': defaultdict(int), 'platform': defaultdict(int)}
dict_storing_information['mouse_single'] = {'assay': defaultdict(int), 'disease': defaultdict(int),
                                            'tissue': defaultdict(int), 'cell type': defaultdict(int), 'platform': defaultdict(int)}

In [15]:
dict_storing_non_labeled_disease_types = {'human':[], 'mouse': [], 'human_single': [], 'mouse_single': []}
dict_storing_non_labeled_tissue_types = {'human':[], 'mouse': [], 'human_single': [], 'mouse_single': []}
dict_storing_mislabeled_diabetes = {'human':[], 'mouse': [], 'human_single': [], 'mouse_single': []}
dict_storing_all_information = {'assay': defaultdict(int), 'disease': defaultdict(int), 'tissue': defaultdict(int), 'platform': defaultdict(int), 'cell type': defaultdict(int)}


In [16]:
dict_to_combine_with_metadata = {'human': {}, 'mouse': {}, 'human_single': {}, 'mouse_single': {}}

In [17]:
list_of_keys_storing_studies = ['human', 'mouse', 'human_single', 'mouse_single']

### Manual Disease Labels

In [18]:
list_storing_manually_annotated_gses = []
mouse_bulk_df = pd.read_csv('mouse_nonlabeled_diseases_manually_labelled.csv', index_col=0)
list_storing_manually_annotated_gses.extend(mouse_bulk_df['GSE'].to_list())
human_bulk_df = pd.read_csv('human_nonlabeled_diseases_manually_labelled.csv', index_col=0)
list_storing_manually_annotated_gses.extend(human_bulk_df['GSE'].to_list())
list_storing_manually_annotated_gses = [x.strip() for x in list_storing_manually_annotated_gses]

## Helper Functions

In [19]:
def match_disease_from_manual_annotations(dict_key_for_studies, value):
    if 'healthy' in value.lower() and 'gene' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Healthy: Gene KO Focus'] +=1
        dict_storing_all_information['disease']['Healthy: Gene KO Focus'] +=1
        return 'Healthy: Gene KO Focus'
    elif 'healthy' in value.lower() and 'treatment' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Healthy: Treatment Focus'] +=1
        dict_storing_all_information['disease']['Healthy: Treatment Focus'] +=1
        return 'Healthy: Treatment Focus'
    elif 'healthy' in value.lower() and 'diet' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Healthy: Diet Focus'] +=1
        dict_storing_all_information['disease']['Healthy: Diet Focus'] +=1
        return 'Healthy: Diet Focus'
    elif 'healthy' in value.lower() and 'age' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Healthy: Age Focus'] +=1
        dict_storing_all_information['disease']['Healthy: Age Focus'] +=1
        return 'Healthy: Age Focus'
    elif 'healthy' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Healthy: No Specific Disease Mention'] +=1
        dict_storing_all_information['disease']['Healthy: No Specific Disease Mention'] +=1
        return 'Healthy: No Specific Disease Mention'
    elif 'nash' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Nonalcoholic Steatohepatitis (NASH)'] +=1
        dict_storing_all_information['disease']['Nonalcoholic Steatohepatitis (NASH)'] +=1
        return 'Nonalcoholic Steatohepatitis (NASH)'
    elif 'nafld' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Non-alcoholic fatty liver disease (NALFD)'] +=1
        dict_storing_all_information['disease']['Non-alcoholic fatty liver disease (NALFD)'] +=1
        return 'Non-alcoholic fatty liver disease (NALFD)'
    elif 'steatosis' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Liver Steatosis'] +=1
        dict_storing_all_information['disease']['Liver Steatosis'] +=1
        return 'Liver Steatosis'
    elif 'obesity' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Obesity'] +=1
        dict_storing_all_information['disease']['Obesity'] +=1
        return 'Obesity'
    elif 'diabetes' in value.lower():
        dict_storing_information[dict_key_for_studies]['disease']['Diabetes (Non-specific)'] +=1
        dict_storing_all_information['disease']['Diabetes (Non-specific)'] +=1
        return 'Diabetes (Non-specific)'
    else:
        dict_storing_information[dict_key_for_studies]['disease'][value] +=1
        dict_storing_all_information['disease'][value] +=1
        return value

In [20]:
def get_clean_tissue(tissue):
    if 'islet' in tissue.lower() or 'pancrea' in tissue.lower() or 'beta cell' in tissue.lower():
        return 'Pancreas'
    elif 'fat' in tissue.lower() or 'adipose' in tissue.lower() or 'adipocyt' in tissue.lower() or 'bat' in tissue.lower():
        return 'Adipose Tissue'
    elif 'breast' in tissue.lower() or 'mammary gland' in tissue.lower():
        return 'Breast'
    elif 'liver' in tissue.lower() or 'hepatoc' in tissue.lower():
        return 'Liver'
    elif 'kidney' in tissue.lower() or 'pancreatic' in tissue.lower():
        return 'Kidney'
    elif 'skin' in tissue.lower():
        return 'Skin'
    elif 'brain' in tissue.lower() or 'cortex' in tissue.lower() or 'hypothalam' in tissue.lower() or 'hippocampus' in tissue.lower():
        return 'Brain'
    elif 'heart' in tissue.lower() or 'cardiac' in tissue.lower() or 'myocard' in tissue.lower():
        return 'Heart'
    elif 'pbmc' in tissue.lower():
        return 'PBMC'
    elif 'skeletal' in tissue.lower():
        return 'Skeletal Muscle'
    elif 'vastus lateral' in tissue.lower() or 'muscle' in tissue.lower() or 'quadr' in tissue.lower():
        return 'Muscle'
    elif 'blood' in tissue.lower():
        return 'Blood'
    elif 'bone' in tissue.lower():
        return 'Bone'
    elif 'stem cell' in tissue.lower():
        return 'Blood'
    elif 'small intestin' in tissue.lower() or 'ileum' in tissue.lower() or 'small in' in tissue.lower() or 'duodenum' in tissue.lower():
        return 'Small Intestine'
    elif 'foxo' in tissue.lower():
        return 'Pancreas'
    elif 'rnv' in tissue.lower() or 'retina' in tissue.lower():
        return 'Retina'
    elif 'vascular' in tissue.lower():
        return 'Blood'
    elif 'thyroid' in tissue.lower():
        return 'Thyroid'
    elif 'esophag' in tissue.lower():
        return 'Esophagus'
    elif 'lung' in tissue.lower():
        return 'Lung'
    elif 'bladder' in tissue.lower():
        return 'Bladder'
    else:
        return tissue

In [21]:
def get_bto_identifier(tissue):
    bto_df = pd.read_csv('bto_identification.csv', index_col=0)
    bto_row = bto_df.loc[bto_df.index == tissue]
    if len(bto_row) > 0:
        tissue_identifier = tissue + ' ' + bto_row.iloc[0]['Identifier']
        return tissue_identifier
    else:
        return tissue


In [22]:
def get_disease_identifier(key):
    diseases_df = pd.read_csv('diseases.tsv', sep='\t', index_col=0)
    key_to_search = key
    if 'Covid' in key:
        key_to_search = 'covid-19'
    if 'Diabetes' in key and 'Non-specific' in key:
        key_to_search = 'diabetes'
    if 'Breast Cancer' in key:
        key_to_search = 'breast cancer'
    if 'Thyroid Cancer' in key:
        key_to_search = 'parathyroid carcinoma'
    if 'Liver Disease' in key:
        key_to_search = 'liver disease'
    if 'Non-alcoholic fatty liver disease' in key:
        key_to_search = 'non-alcoholic fatty liver disease'
    if 'Liver Cancer' in key:
        key_to_search = 'hepatocellular carcinoma'
    if 'Pulmonary Hypertension' in key:
        key_to_search =  'pulmonary hypertension'
    doid_row = diseases_df.loc[(diseases_df['Disease'].str.contains(key_to_search.lower()))]
    if len(doid_row) > 0:
        disease_identifier = key + ' ' + doid_row.iloc[0]['DOID']
        return disease_identifier
    else:
        return key


## Full study annotation

In [ ]:
for key in ['human', 'mouse']:
    studies = dict_storing_gses[key]
    if 'human' in key:
        geo_organism = 'Homo sapiens'
    if 'mouse' in key:
        geo_organism = 'Mus musculus'
    path_to_study_folder = './studies/' + key
    if not os.path.exists(path_to_study_folder):
        os.makedirs(path_to_study_folder)
        
    dict_key_for_studies = key
    for study in studies:
        dict_to_combine_with_metadata[dict_key_for_studies][study] = {}     
        gse = GEOparse.get_GEO(geo = study, destdir = f'{path_to_study_folder}/{study}', silent=True)
        # ASSAY TYPE CLASSIFICATION
        if 'type' in gse.metadata.keys():
            for assay in gse.metadata['type']:
                if 'Expression profiling by high throughput sequencing' in assay or 'Genome binding/occupancy profiling by high throughput sequencing' in assay or 'Non-coding RNA profiling by high throughput sequencing' in assay:
                    dict_storing_information[dict_key_for_studies]['assay']['RNA seq'] +=1
                    dict_storing_all_information['assay']['RNA seq'] +=1
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['assay'] = 'RNA seq'
                    break
                elif 'Expression profiling by array' in assay:
                    dict_storing_information[dict_key_for_studies]['assay']['Microarray'] +=1
                    dict_storing_all_information['assay']['Microarray'] +=1
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['assay'] = 'Microarray'
                    break
                else:
                    dict_storing_information[dict_key_for_studies]['assay']['RNA seq'] +=1
                    dict_storing_all_information['assay']['RNA seq'] +=1
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['assay'] = 'RNA seq'
                    print(assay)
                    break

        ## PLATFORM ID CLASSIFICATION
        if 'platform_id' in gse.metadata.keys():
            for platform in gse.metadata['platform_id']:
                dict_storing_information[dict_key_for_studies]['platform'][platform] +=1
                dict_storing_all_information['platform'][platform] +=1
                dict_to_combine_with_metadata[dict_key_for_studies][study]['platform'] = platform
                break
        else:
            dict_storing_information[dict_key_for_studies]['platform']['No Platform information'] +=1
            dict_storing_all_information['platform']['No Platform information'] +=1
            dict_to_combine_with_metadata[dict_key_for_studies][study]['platform'] = 'No Platform information'
        
        # DISEASE CLASSIFICATION
        string_to_concatenate = ''
        if 'summary' in gse.metadata.keys() and 'title' in gse.metadata.keys():
            for title in gse.metadata['title']:
                title = ftfy.fix_text(title)
                string_to_concatenate += title
            for summary in gse.metadata['summary']:
                summary = ftfy.fix_text(summary)
                string_to_concatenate += ' '+ summary
        
            string_to_concatenate = string_to_concatenate.lower()

            if 't1d' in string_to_concatenate or "type 1 diabetes" in string_to_concatenate or 'type-1-diabetes' in string_to_concatenate:
                dict_storing_information[dict_key_for_studies]['disease']['Type 1 Diabetes'] += 1
                dict_storing_all_information['disease']['Type 1 Diabetes'] += 1
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = 'Type 1 Diabetes'
                disease_with_identifier = get_disease_identifier('Type 1 Diabetes')
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier
            elif 't2d' in string_to_concatenate or 'type 2 diabetes' in string_to_concatenate:
                dict_storing_information[dict_key_for_studies]['disease']['Type 2 Diabetes'] += 1
                dict_storing_all_information['disease']['Type 2 Diabetes'] += 1
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = 'Type 2 Diabetes'
                disease_with_identifier = get_disease_identifier('Type 2 Diabetes')
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier
            elif 'diabetes' in string_to_concatenate or 'diabetic' in string_to_concatenate:
                dict_storing_information[dict_key_for_studies]['disease']['Diabetes (Non-specific)'] += 1
                dict_storing_all_information['disease']['Diabetes (Non-specific)'] += 1
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = 'Diabetes (Non-specific)'
                disease_with_identifier = get_disease_identifier('Diabetes (Non-specific)')
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier
            elif 'pancreas' in string_to_concatenate or 'pancreatic' in string_to_concatenate or 'islet' in string_to_concatenate or 'β-cell' in string_to_concatenate or 'β cell' in string_to_concatenate:
                dict_storing_information[dict_key_for_studies]['disease']['Diabetes (Non-specific)'] += 1
                dict_storing_all_information['disease']['Diabetes (Non-specific)'] += 1
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = 'Diabetes (Non-specific)'
                dict_storing_mislabeled_diabetes[dict_key_for_studies].append(study)
                disease_with_identifier = get_disease_identifier('Diabetes (Non-specific)')
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier
            #Using the manually labelled data to fill in the nondiabetes related information
            else:
                if dict_key_for_studies == 'mouse':
                    mouse_bulk_df = pd.read_csv('mouse_nonlabeled_diseases_manually_labelled.csv', index_col=0)
                    row_for_study = mouse_bulk_df.loc[mouse_bulk_df['GSE'] == study]
                    if len(row_for_study) > 0:
                        value = row_for_study.iloc[0]['Manual Label']
                        disease_returned = match_disease_from_manual_annotations(dict_key_for_studies, value)
                        dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = disease_returned
                        disease_with_identifier = get_disease_identifier(disease_returned)
                        dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier
                elif dict_key_for_studies == 'human':
                    human_bulk_df = pd.read_csv('human_nonlabeled_diseases_manually_labelled.csv', index_col=0)
                    row_for_study = human_bulk_df.loc[human_bulk_df['GSE'] == study]
                    if len(row_for_study) > 0:
                        value = row_for_study.iloc[0]['Manual Label']
                        disease_returned = match_disease_from_manual_annotations(dict_key_for_studies, value)
                        dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = disease_returned
                        disease_with_identifier = get_disease_identifier(disease_returned)
                        dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier
                else:
                    dict_storing_information[dict_key_for_studies]['disease']['Non Diabetes Related'] += 1
                    dict_storing_all_information['disease']['Non Diabetes Related'] += 1
                    dict_storing_non_labeled_disease_types[dict_key_for_studies].append(study)
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = 'Non Diabetes Related'
                    disease_with_identifier = get_disease_identifier('Non Diabetes Related')
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier



        # TISSUE AND CELL TYPE CLASSIFICATION   
        # Used to determine the tissue type using the manually annotated ones and if not the manual annotated go into the the gse sample metadata
        if study.strip() in list_storing_manually_annotated_gses:
            if dict_key_for_studies == 'mouse':
                mouse_bulk_df = pd.read_csv('mouse_nonlabeled_diseases_manually_labelled.csv', index_col=0)
                row_for_study = mouse_bulk_df.loc[mouse_bulk_df['GSE'] == study]
                if len(row_for_study) > 0:
                    tissue_val = row_for_study.iloc[0]['Tissue Type']
                    dict_storing_information[dict_key_for_studies]['tissue'][tissue_val] += 1
                    dict_storing_all_information['tissue'][tissue_val] += 1
                    cleaned_tissue = get_clean_tissue(tissue_val)
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type'] = cleaned_tissue
                    tissue_with_identifier = get_bto_identifier(cleaned_tissue)
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type_identifier'] = tissue_with_identifier
            elif dict_key_for_studies == 'human':
                human_bulk_df = pd.read_csv('human_nonlabeled_diseases_manually_labelled.csv', index_col=0)
                row_for_study = human_bulk_df.loc[human_bulk_df['GSE'] == study]
                if len(row_for_study) > 0:
                    tissue_val = row_for_study.iloc[0]['Tissue Type']
                    dict_storing_information[dict_key_for_studies]['tissue'][tissue_val] += 1
                    dict_storing_all_information['tissue'][tissue_val] += 1
                    cleaned_tissue = get_clean_tissue(tissue_val)
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type'] = cleaned_tissue
                    tissue_with_identifier = get_bto_identifier(cleaned_tissue)
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type_identifier'] = tissue_with_identifier
        else:
            samples = gse.metadata['sample_id']
            for samp in samples:
                conditions =  gse.gsms[samp].metadata['characteristics_ch1']
                tissue_var = False
                cell_type_var = False
                for cond in conditions:
                    if ':' in cond:
                        key, value = cond.split(":")[0].strip(), cond.split(":", 1)[1].strip()
                        if 'tissue' in key:
                            tissue = value.strip().lower()
                            dict_storing_information[dict_key_for_studies]['tissue'][tissue] += 1
                            dict_storing_all_information['tissue'][tissue] += 1
                            tissue_var = True
                            cleaned_tissue = get_clean_tissue(tissue)
                            dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type'] = cleaned_tissue
                            tissue_with_identifier = get_bto_identifier(cleaned_tissue)
                            dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type_identifier'] = tissue_with_identifier
                            break
                if not tissue_var:
                    if dict_key_for_studies == 'human' and study.strip() not in list_storing_manually_annotated_gses:
                        human_tissue_df = pd.read_csv('human_nonlabeled_tissues_manually_labelled.csv', index_col=0)
                        row_for_study = human_tissue_df.loc[human_tissue_df['GSE'] == study]
                        if len(row_for_study) > 0:
                            tissue_val = row_for_study.iloc[0]['Tissue Type']
                            dict_storing_information[dict_key_for_studies]['tissue'][tissue_val] += 1
                            dict_storing_all_information['tissue'][tissue_val] += 1
                            cleaned_tissue = get_clean_tissue(tissue_val)
                            dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type'] = cleaned_tissue
                            tissue_with_identifier = get_bto_identifier(cleaned_tissue)
                            dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type_identifier'] = tissue_with_identifier
                    if dict_key_for_studies == 'mouse' and study.strip() not in list_storing_manually_annotated_gses:
                        mouse_tissue_df = pd.read_csv('mouse_nonlabeled_tissues_manually_labelled.csv', index_col=0)
                        row_for_study = mouse_tissue_df.loc[mouse_tissue_df['GSE'] == study]
                        if len(row_for_study) > 0:
                            tissue_val = row_for_study.iloc[0]['Tissue Type']
                            dict_storing_information[dict_key_for_studies]['tissue'][tissue_val] += 1
                            dict_storing_all_information['tissue'][tissue_val] += 1
                            cleaned_tissue = get_clean_tissue(tissue_val)
                            dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type'] = cleaned_tissue
                            tissue_with_identifier = get_bto_identifier(cleaned_tissue)
                            dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type_identifier'] = tissue_with_identifier

                break


In [ ]:
for key in ['human_single', 'mouse_single']:
    single_species_full_info_df = pd.read_csv('{}_all_studies_fully_labelled.csv'.format(key), index_col=0)

    studies = dict_storing_gses[key]
    if 'human' in key:
        geo_organism = 'Homo sapiens'
    if 'mouse' in key:
        geo_organism = 'Mus musculus'
    path_to_study_folder = './studies/' + key
    if not os.path.exists(path_to_study_folder):
        os.makedirs(path_to_study_folder)
    dict_key_for_studies = key
    for study in studies:
        dict_to_combine_with_metadata[dict_key_for_studies][study] = {}     
        gse = GEOparse.get_GEO(geo = study, destdir = f'{path_to_study_folder}/{study}', silent=True)
        # ASSAY TYPE CLASSIFICATION
        if 'type' in gse.metadata.keys():
            for assay in gse.metadata['type']:
                if 'Expression profiling by high throughput sequencing' in assay or 'Genome binding/occupancy profiling by high throughput sequencing' in assay or 'Non-coding RNA profiling by high throughput sequencing' in assay:
                    dict_storing_information[dict_key_for_studies]['assay']['RNA seq'] +=1
                    dict_storing_all_information['assay']['RNA seq'] +=1
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['assay'] = 'RNA seq'
                    break
                elif 'Expression profiling by array' in assay:
                    dict_storing_information[dict_key_for_studies]['assay']['Microarray'] +=1
                    dict_storing_all_information['assay']['Microarray'] +=1
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['assay'] = 'Microarray'
                    break
                else:
                    dict_storing_information[dict_key_for_studies]['assay']['RNA seq'] +=1
                    dict_storing_all_information['assay']['RNA seq'] +=1
                    dict_to_combine_with_metadata[dict_key_for_studies][study]['assay'] = 'RNA seq'
                    print(assay)
                    break

        ## PLATFORM ID CLASSIFICATION
        if 'platform_id' in gse.metadata.keys():
            for platform in gse.metadata['platform_id']:
                dict_storing_information[dict_key_for_studies]['platform'][platform] +=1
                dict_storing_all_information['platform'][platform] +=1
                dict_to_combine_with_metadata[dict_key_for_studies][study]['platform'] = platform
                break
        else:
            dict_storing_information[dict_key_for_studies]['platform']['No Platform information'] +=1
            dict_storing_all_information['platform']['No Platform information'] +=1
            dict_to_combine_with_metadata[dict_key_for_studies][study]['platform'] = 'No Platform information'
        
        # DISEASE and Tissue CLASSIFICATION
        row_for_study = single_species_full_info_df.loc[single_species_full_info_df['GSE'] == study]
        if len(row_for_study) > 0:
            disease = row_for_study.iloc[0]['Disease Label']
            tissue = row_for_study.iloc[0]['Tissue']
            cell_type = row_for_study.iloc[0]['Cell Type']
            if 'diabetes' in disease.lower():
                dict_storing_information[dict_key_for_studies]['disease']['Diabetes (Non-specific)'] +=1
                dict_storing_all_information['disease']['Diabetes (Non-specific)'] +=1
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = 'Diabetes (Non-specific)'
                disease_with_identifier = get_disease_identifier('Diabetes (Non-specific)')
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier
            elif 'healthy' in disease.lower():
                dict_storing_information[dict_key_for_studies]['disease']['Healthy: No Specific Disease Mention'] +=1
                dict_storing_all_information['disease']['Healthy: No Specific Disease Mention'] +=1
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = 'Healthy: No Specific Disease Mention'
                disease_with_identifier = get_disease_identifier('Healthy: No Specific Disease Mention')
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier
            else:
                dict_storing_information[dict_key_for_studies]['disease'][disease] += 1
                dict_storing_all_information['disease'][disease] += 1
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease'] = disease
                disease_with_identifier = get_disease_identifier(disease)
                dict_to_combine_with_metadata[dict_key_for_studies][study]['disease_type_identifier'] = disease_with_identifier

            # Tissue type
            dict_storing_information[dict_key_for_studies]['tissue'][tissue] += 1
            dict_storing_all_information['tissue'][tissue] += 1
            cleaned_tissue = get_clean_tissue(tissue)
            dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type'] = cleaned_tissue
            tissue_with_identifier = get_bto_identifier(cleaned_tissue)
            dict_to_combine_with_metadata[dict_key_for_studies][study]['tissue_type_identifier'] = tissue_with_identifier

            # Cell Type
            dict_storing_information[dict_key_for_studies]['cell type'][cell_type] += 1
            dict_storing_all_information['cell type'][cell_type] += 1
            dict_to_combine_with_metadata[dict_key_for_studies][study]['cell type'] = cell_type




## Count Confirmation

In [25]:
print('Total number of each study type')
print(len(dict_storing_gses['human']))
print(len(dict_storing_gses['mouse']))
print(len(dict_storing_gses['human_single']))
print(len(dict_storing_gses['mouse_single']))
print('Number of Assay Measurements')
print(sum(map(lambda x : x,dict_storing_information['human']['assay'].values())))
print(sum(map(lambda x : x,dict_storing_information['human_single']['assay'].values())))
print(sum(map(lambda x : x,dict_storing_information['mouse']['assay'].values())))
print(sum(map(lambda x : x,dict_storing_information['mouse_single']['assay'].values())))
print('Number of Platform Measurements')
print(sum(map(lambda x : x,dict_storing_information['human']['platform'].values())))
print(sum(map(lambda x : x,dict_storing_information['human_single']['platform'].values())))
print(sum(map(lambda x : x,dict_storing_information['mouse']['platform'].values())))
print(sum(map(lambda x : x,dict_storing_information['mouse_single']['platform'].values())))
print('Number of Disease Measurements')
print(sum(map(lambda x : x,dict_storing_information['human']['disease'].values())))
print(sum(map(lambda x : x,dict_storing_information['human_single']['disease'].values())))
print(sum(map(lambda x : x,dict_storing_information['mouse']['disease'].values())))
print(sum(map(lambda x : x,dict_storing_information['mouse_single']['disease'].values())))
print('Total number of tissue types determined')
print(sum(map(lambda x : x,dict_storing_information['human']['tissue'].values())))
print(sum(map(lambda x : x,dict_storing_information['human_single']['tissue'].values())))
print(sum(map(lambda x : x,dict_storing_information['mouse']['tissue'].values())))
print(sum(map(lambda x : x,dict_storing_information['mouse_single']['tissue'].values())))
print('Total number of tissue type classifications determined')
print(len(dict_storing_information['human']['tissue']))
print(len(dict_storing_information['mouse']['tissue']))
print(len(dict_storing_information['human_single']['tissue']))
print(len(dict_storing_information['mouse_single']['tissue']))
# print('Total number of consolidated tissue types determined')
# print(len(dict_storing_information['human']['tissue_cleaned']))
# print(len(dict_storing_information['mouse']['tissue_cleaned']))
# print(len(dict_storing_information['human_single']['tissue_cleaned']))
# print(len(dict_storing_information['mouse_single']['tissue_cleaned']))

Total number of each study type
145
145
17
30
Number of Assay Measurements
145
17
145
30
Number of Platform Measurements
145
17
145
30
Number of Disease Measurements
145
17
145
30
Total number of tissue types determined
145
17
145
30
Total number of tissue type classifications determined
83
58
7
12


# WHOLE DATASET CLASSIFCATION ANNOTATIONS

In [26]:
#Consolidating Certain Terms Manually as of now

dict_storing_all_information['tissue_cleaned'] = defaultdict(int)
for tissue, count in dict_storing_all_information['tissue'].items():
    if 'islet' in tissue.lower() or 'pancrea' in tissue.lower() or 'beta cell' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Pancreas'] += count
    elif 'fat' in tissue.lower() or 'adipose' in tissue.lower() or 'adipocyt' in tissue.lower() or 'bat' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Adipose Tissue'] += count
    elif 'breast' in tissue.lower() or 'mammary gland' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Breast'] += count
    elif 'liver' in tissue.lower() or 'hepatoc' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Liver'] += count
    elif 'kidney' in tissue.lower() or 'pancreatic' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Kidney'] += count
    elif 'skin' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Skin'] += count
    elif 'brain' in tissue.lower() or 'cortex' in tissue.lower() or 'hypothalam' in tissue.lower() or 'hippocampus' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Brain'] += count
    elif 'heart' in tissue.lower() or 'cardiac' in tissue.lower() or 'myocard' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Heart'] += count
    elif 'pbmc' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['PBMC'] += count
    elif 'skeletal' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Skeletal Muscle'] += count
    elif 'vastus lateral' in tissue.lower() or 'muscle' in tissue.lower() or 'quadr' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Muscle'] += count
    elif 'blood' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Blood'] += count
    elif 'bone' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Bone'] += count
    elif 'stem cell' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Blood'] += count
    elif 'small intestin' in tissue.lower() or 'ileum' in tissue.lower() or 'small in' in tissue.lower() or 'duodenum' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Small Intestine'] += count
    elif 'foxo' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Pancreas'] += count
    elif 'rnv' in tissue.lower() or 'retina' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Retina'] += count
    elif 'vascular' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Blood'] += count
    elif 'thyroid' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Thyroid'] += count
    elif 'esophag' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Esophagus'] += count
    elif 'lung' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Lung'] += count
    elif 'bladder' in tissue.lower():
        dict_storing_all_information['tissue_cleaned']['Bladder'] += count
    else:
        dict_storing_all_information['tissue_cleaned'][tissue] += count

In [27]:
#Sorting the data and creating legend for pie charts
legend_dict_full = defaultdict()

for metadata_key, dictionary_unsorted in dict_storing_all_information.items():
    dictionary_sorted = sorted(dict_storing_all_information[metadata_key].items(), key = lambda x:x[1], reverse=True)
    new_dict = {}
    for item in dictionary_sorted:
        new_dict[item[0]] = item[1]
    dict_storing_all_information[metadata_key] = new_dict
    
    if 'tissue' in metadata_key or 'disease' in metadata_key:
        temp_dict = {}
        for key, value in dict_storing_all_information[metadata_key].items():
            percent = str( round(float(value/347 *100), 2) ) + '%'
            temp_dict[key] = key + " " + percent
        legend_dict_full[metadata_key] = temp_dict

## Output Data Generation files

In [ ]:
for species, dictionary_of_gses in dict_to_combine_with_metadata.items():
    print(species)
    for gse, metadata in dictionary_of_gses.items():
        print(gse)
        metadata_file = base_url + '/' + species + '/' + gse + '/' + gse + '_Metadata.tsv'
        metadata_dataframe = pd.read_csv(s3.open(metadata_file), sep='\t')
        list_of_metadata_perts = metadata_dataframe.iloc[:,1].unique().tolist()
        list_of_metadata_perts = list(set([x.strip() for x in list_of_metadata_perts]))
        dict_to_combine_with_metadata[species][gse]['perturbations'] = list_of_metadata_perts

In [ ]:
dict_to_combine_with_metadata

In [30]:
import pickle
with open('./additional_metadata_d2h2.pickle', 'wb') as f:
	pickle.dump(dict_to_combine_with_metadata, f, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
# Create the counts json for each of the metadata fields that will be used to make the interactve plots on the frontend.
dict_for_json = {}
dict_for_json['assay'] = {'labels':list(dict_storing_all_information['assay'].keys()), 'counts': list(dict_storing_all_information['assay'].values())}
dict_for_json['disease'] = {'labels':list(dict_storing_all_information['disease'].keys()), 'counts': list(dict_storing_all_information['disease'].values())}
dict_for_json['platform'] = {'labels':list(dict_storing_all_information['platform'].keys()), 'counts': list(dict_storing_all_information['platform'].values())}
dict_for_json['cell type'] = {'labels':list(dict_storing_all_information['cell type'].keys()), 'counts': list(dict_storing_all_information['cell type'].values())}
dict_for_json['tissue'] = {'labels':list(dict_storing_all_information['tissue_cleaned'].keys()), 'counts': list(dict_storing_all_information['tissue_cleaned'].values())}
for key, dict in dict_for_json.items():
    print(key)
    print(len(dict_for_json[key]['labels']))

with open('metadata_counts.json', 'w') as f:
    json.dump(dict_for_json, f)

assay
2
disease
29
platform
29
cell type
18
tissue
31
